In [1]:
import gym
import gym_Snake
import time
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from multiprocessing import Pool

In [2]:
nb_tests_file = 50

In [3]:
import multiprocessing.pool

class NoDaemonProcess(multiprocessing.Process):
    @property
    def daemon(self):
        return False

    @daemon.setter
    def daemon(self, value):
        pass


class NoDaemonContext(type(multiprocessing.get_context())):
    Process = NoDaemonProcess

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class NestablePool(multiprocessing.pool.Pool):
    def __init__(self, *args, **kwargs):
        kwargs['context'] = NoDaemonContext()
        super(NestablePool, self).__init__(*args, **kwargs)


In [4]:
reward_eat = 10

custom_rewards = {
    "REWARD_TARGET": reward_eat,
    "REWARD_COLLISION": -100,
    "REWARD_TOWARD": 1,
    "REWARD_AWAY": 0
}

env_name = 'Double_v'

env = gym.make('Snake-v0', 
               player = 'computer', 
               shape = env_name, 
               state_mode = 'states', 
               reward_mode = 'normal', 
               width = 10, 
               height = 10, 
               solid_border = True,
               rewards = custom_rewards)

In [5]:
def pool_tree_search_best_action(env, dept, target_reward, pooling_condition=np.inf):
    actions = [0, 1, 2]
    envs = [env.clone() for _ in actions]
    
    # Play each action
    obss, rewards, dones, _ = np.array([e.step(a) for e,a in zip(envs, actions)]).T
    
    
    with NestablePool(3) as p:
        p.daemon = False
        res = p.starmap(pool_tree_search_best_action_rec, [[envs[i], dept - 1, [i], target_reward, pooling_condition] for i in actions])
        results, act = np.array(res).T

        
        return np.argmax([0.9 * rs + rw for rs, rw in zip(results, rewards)])

In [6]:
def tree_search_best_action(env, dept, target_reward):
    global stop
    stop = False
    results, act = tree_search_best_action_rec(env, dept, acts=[], target_reward=target_reward)
#     print(results)
    return act

In [7]:
# stop as soon as a path can reach the end without dying
def check_trap(env, dept):
    pass

In [8]:
stop = False
def pool_tree_search_best_action_rec(env, dept, acts, target_reward, pooling_condition=np.inf, dept_after_reward=4):
    global stop
    actions = [0, 1, 2]
    
    # Clone envs
    envs = [env.clone() for _ in actions]
    
    # Play each action
    obss, rewards, dones, _ = np.array([e.step(a) for e,a in zip(envs, actions)]).T

    # Termination case
    if dept == 0:
        valids = [i for i, x in enumerate(rewards) if x == max(rewards)]
        i = np.random.choice(valids)
        return rewards[i], i
    
    remaining_dept = [dept - 1 for _ in actions]
    
    # Mark as done loops that have already reached the target
    # Allows to slightly accellerate the algorithm, but suffers from the same
    # problems of Q-learning and SARSA (snake traps itself)
#     for i, r in enumerate(rewards):
#         if r >= target_reward:
#             dones = [True for _ in range(len(dones))] # Stop search for this branch
#             stop = True # Send the stop signal to all branches
#             dones[i] = True # Stop search for given path
#             remaining_dept = [0 for _ in actions] # Stop search for other paths, limit for given one
#             remaining_dept[i] = min(dept_after_reward, remaining_dept[i]) # Limit the search for the given path
         
    # Check if external stop - makes it very fast limiting dept in most cases
    if stop:
        dones = [True for _ in range(len(dones))]
            
    
    # Multiprocess if still high in tree
#     if pooling_condition(dept):
    if dept > pooling_condition:
        with NestablePool(3) as p:
            p.daemon = False
            res = p.starmap(pool_tree_search_best_action_rec, [[envs[i], dept - 1, acts + [i], target_reward, pooling_condition] for i in actions])
            results, act = np.array(res).T
    else:
        results, _ = np.array([tree_search_best_action_rec(e, dept-1, acts + [i], target_reward=target_reward) if not dones[i] else (0, -1) for i, e, d in zip(range(len(actions)), envs, remaining_dept)]).T

#     results, _ = np.array([tree_search_best_action_rec(e, dept-1, acts + [i], target_reward=target_reward) if not dones[i] else (0, -1) for i, e in enumerate(envs)]).T
#     results = [tree_search_best_action_rec(e, dept-1, acts + [i]) for i, e in enumerate(envs)]


    # Add reward MULTIPLIED by dept. In this way, sort of discount rewards more far away
    # Without this trick, it is possible that the snake turn around the target without ever eating it, since a 
    # position in which we can eat the target in N steps has the same value as the one in which we eat it
    results = [results[i] + rewards[i] * (1 + dept/1000) for i in range(len(results))]
#     results = [results[i] + rewards[i] for i in range(len(results))]

    
    # If multiple results with same (max) value, choose random among them 
    valids = [i for i, x in enumerate(results) if x == max(results)]
    
    # Randomly choose one of the best actions
    i = np.random.choice(valids)
    
#     if dept == 5: print(results)
    return results[i], i

In [9]:
stop = False
def tree_search_best_action_rec(env, dept, acts, target_reward, dept_after_reward=4):
    global stop
    actions = [0, 1, 2]
    
    # Clone envs
    envs = [env.clone() for _ in actions]
    
    # Play each action
    obss, rewards, dones, _ = np.array([e.step(a) for e,a in zip(envs, actions)]).T

    # Termination case
    if dept == 0:
        valids = [i for i, x in enumerate(rewards) if x == max(rewards)]
        i = np.random.choice(valids)
        return rewards[i], i
    
    remaining_dept = [dept - 1 for _ in actions]
    
    # Mark as done loops that have already reached the target
    # Allows to slightly accellerate the algorithm, but suffers from the same
    # problems of Q-learning and SARSA (snake traps itself)
#     for i, r in enumerate(rewards):
#         if r >= target_reward:
#             dones = [True for _ in range(len(dones))] # Stop search for this branch
#             stop = True # Send the stop signal to all branches
#             dones[i] = True # Stop search for given path
#             remaining_dept = [0 for _ in actions] # Stop search for other paths, limit for given one
#             remaining_dept[i] = min(dept_after_reward, remaining_dept[i]) # Limit the search for the given path
         
    # Check if external stop - makes it very fast limiting dept in most cases
    if stop:
        dones = [True for _ in range(len(dones))]
            
    
    results, _ = np.array([tree_search_best_action_rec(e, dept-1, acts + [i], target_reward=target_reward) if not dones[i] else (0, -1) for i, e, d in zip(range(len(actions)), envs, remaining_dept)]).T
#     results, _ = np.array([tree_search_best_action_rec(e, dept-1, acts + [i], target_reward=target_reward) if not dones[i] else (0, -1) for i, e in enumerate(envs)]).T
#     results = [tree_search_best_action_rec(e, dept-1, acts + [i]) for i, e in enumerate(envs)]


    # Add reward MULTIPLIED by dept. In this way, sort of discount rewards more far away
    # Without this trick, it is possible that the snake turn around the target without ever eating it, since a 
    # position in which we can eat the target in N steps has the same value as the one in which we eat it
    results = [results[i] + rewards[i] * (1 + dept/1000) for i in range(len(results))]
#     results = [results[i] + rewards[i] for i in range(len(results))]

    
    # If multiple results with same (max) value, choose random among them 
    valids = [i for i, x in enumerate(results) if x == max(results)]
    
    # Randomly choose one of the best actions
    i = np.random.choice(valids)
    
#     if dept == 5: print(results)
    return results[i], i

In [10]:
def play_epoch(env, render = False, dept=5, target_reward = 10, sleep_time = 0.5, max_step = 100, pooling_condition = np.inf):
    
    # Reset env
    obs = env.reset()

    done = False
    
    # Sum the rewards
    total_rew = 0
    total_eated = 0
    
    i = 0
    while not done:
        # Show
        if render: env.render()
        # Choose next action
#         new_act = tree_search_best_action(env, dept=dept, target_reward=target_reward)
        new_act = pool_tree_search_best_action(env, dept=dept, target_reward=target_reward, pooling_condition=pooling_condition)
#         print(new_act)
        # Act in the env
        obs, reward, done, info = env.step(new_act)
        # Count eat
        if reward == reward_eat: total_eated += 1
        # Store reward
        total_rew += reward
        # Slow render
        if render and sleep_time > 0: time.sleep(sleep_time)
        i += 1
        if i == max_step: break
            
    # Return total reward
    return total_rew, total_eated, i

In [11]:
# i = 7
# # cond = lambda x : x > 5
# cond = 5
# start = time.time()
# r, e, i = play_epoch(env = env, render = False, dept=i, target_reward=10, sleep_time = 0, max_step = 1000, pooling_condition = cond)
# stop = time.time()
# print(f'i = {i}, total reward = {r}, total eat = {e}, time = {round(stop-start, 2)}')

In [12]:
# Test

i = 7
cond = 5

nb_tests = 1

times = []
times_per_step = []
eated = []


# for _ in tqdm(range(nb_tests)):
start = time.time()
r, e, i = play_epoch(env = env, render = False, dept=i, target_reward=10, sleep_time = 0, max_step = 1000, pooling_condition = cond)
stop = time.time()
times.append(stop - start)
times_per_step.append((stop - start) / i)
eated.append(e)
    
# print('\n#########################')
# print(f'After {nb_tests} tests:')
# print(f'Average rewards eated:       {np.mean(eated)}')
# print(f'Max rewards eated:           {np.max(eated)}')
# print(f'Median of rewards eated:     {np.median(eated)}')
# print(f'Average time per simulation: {np.mean(times)}')
# print(f'Average time per step: {np.sum(times) / np.sum(i)}')

# Evaluation problem: all system crashes if play_epoch is called more then once in the same kernel. Probably the processes spawned are not closed correctly. To run multiple test, thus run this notebook multiple time and save intermediate results to external file. Function at the end of the file allows to automatically restart and run all

#### Create and save first, empty df. Comment before continuing

In [13]:
# df = pd.DataFrame(columns=['Average_rewards_eated', 'Max_rewards_eated', 'Median_of_rewards_eated', 'Std_of_rewards_eated', 'Average_time_per_simulation', 'Average_time_per_step'])

In [14]:
# df.to_csv(f'TreeSearch_results/TreeSearchResults_{env_name}.csv', index=False)

## Load df. Should already exist the first time

In [15]:
loaded_df = pd.read_csv(f'TreeSearch_results/TreeSearchResults_{env_name}.csv', index_col=False)

## Get the new results

In [16]:
new_vals = {'Average_rewards_eated': [np.mean(eated)],
            'Max_rewards_eated': [np.max(eated)],
            'Median_of_rewards_eated': [np.median(eated)],
            'Std_of_rewards_eated': [np.std(eated)],
            'Average_time_per_simulation': [np.mean(times)],
            'Average_time_per_step': [np.sum(times) / np.sum(i)]
           }

In [17]:
# loaded_df

## Insert the new results in the df

In [18]:
new_df = pd.concat([loaded_df, pd.DataFrame(new_vals)], ignore_index=True)

In [19]:
# new_df

## Save the new df (with the new results included)

In [20]:
new_df.to_csv(f'TreeSearch_results/TreeSearchResults_{env_name}.csv', index=False)

In [21]:
len(new_df)

50

## If necessary, restart

In [22]:
from IPython.display import HTML, Javascript, display
def restart_kernel_and_run_all_cells():
    display(HTML(
        '''
            <script>
                code_show = false;
                function restart_run_all(){
                    IPython.notebook.kernel.restart();
                    setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    }, 10000)
                }
                function code_toggle() {
                    if (code_show) {
                        $('div.input').hide(200);
                    } else {
                        $('div.input').show(200);
                    }
                    code_show = !code_show
                }
                code_toggle() 
                restart_run_all()
            </script>

        '''
    ))
    
if len(new_df) < nb_tests_file:
    print(len(new_df))
    restart_kernel_and_run_all_cells()

In [27]:
df = pd.read_csv(f'TreeSearch_results/TreeSearchResults_{env_name}.csv', index_col=False)

In [28]:
df

,Average_rewards_eated,Max_rewards_eated,Median_of_rewards_eated,Std_of_rewards_eated,Average_time_per_simulation,Average_time_per_step
0,16.0,16,16.0,0.0,28.166911,0.176043
1,22.0,22,22.0,0.0,46.466373,0.191220
2,19.0,19,19.0,0.0,41.858948,0.202217
3,17.0,17,17.0,0.0,33.534891,0.210911
4,16.0,16,16.0,0.0,34.862572,0.207515
5,12.0,12,12.0,0.0,22.732352,0.210485
6,17.0,17,17.0,0.0,36.478376,0.197180
7,17.0,17,17.0,0.0,30.248036,0.207178
8,22.0,22,22.0,0.0,47.266448,0.204617
9,19.0,19,19.0,0.0,29.707473,0.185672


In [29]:
print('\n#########################')
print(f'After {nb_tests} tests in the {env_name} env:')
print(f'Average rewards eated:       {np.mean(df["Average_rewards_eated"])}')
print(f'Max rewards eated:           {np.max(df["Max_rewards_eated"])}')
print(f'Median of rewards eated:     {np.median(df["Max_rewards_eated"])}')
print(f'Std of rewards eated:        {np.std(df["Max_rewards_eated"])}')
print(f'Average time per simulation: {np.mean(df["Average_time_per_step"])}')  # Not perfectly correct since some are over a bigger number of steps. To be fully correct we should rather save the nb of steps, but the difference is negligiable
print(f'Average time per step:       {np.sum(times) / np.sum(i)}')
print(f'Max time per step:           {np.max(df["Average_time_per_step"])}')


#########################
After 1 tests in the Double_v env:
Average rewards eated:       16.16
Max rewards eated:           27
Median of rewards eated:     15.0
Std of rewards eated:        3.5853033344474494
Average time per simulation: 0.20689094283646725
Average time per step:       0.20391118853059534
Max time per step:           0.2339633909383214


In [26]:
# print(len(new_df))